In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

### Remove the File from folder

In [0]:
dbutils.fs.rm("dbfs:/FileStore/Datasets/chekpoint/",True)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:718)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:437)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:437)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

In [0]:
%sql
create schema bronze;

## Customer_Table

In [0]:
customer_schema =StructType([
    StructField("customer_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("address", StringType(), True),
    StructField("credit_score", IntegerType(), True),
    StructField("join_date", DateType(), True),
    StructField("last_update", TimestampType(), True)
])   

customer_df=spark.read.csv("dbfs:/FileStore/capstone/dataset/customers.csv", header=True, schema=customer_schema)
customer_df.write.format("delta").mode("append").saveAsTable("bronze.customers")


### Branch Table

In [0]:
branches_schema = StructType([
    StructField("branch_id", StringType(), True),
    StructField("branch_name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("timezone", StringType(), True)
])

branch_df=spark.read.csv("dbfs:/FileStore/capstone/dataset/branches.csv", header=True, schema=branches_schema)
branch_df.write.format("delta").mode("append").saveAsTable("bronze.branches")
branch_df.display()

branch_id,branch_name,location,timezone
B0000,North Branch,Michaelshire,PST
B0001,East Branch,Trantown,GMT
B0002,Central Branch,Burtonburgh,EST
B0003,East Branch,Andersonmouth,EST
B0004,West Branch,Port Joseph,AEST
B0005,Downtown Branch,West Nathanland,GMT
B0006,West Branch,New Jameschester,GMT
B0007,West Branch,Catherinechester,PST
B0008,West Branch,Brownfort,GMT
B0009,North Branch,Port Henryshire,PST


### Streamming Data from transaction table

In [0]:

transactions_schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("branch_id", IntegerType(), True),
    StructField("channel", StringType(), True),
    StructField("transaction_type", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("currency", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("status", StringType(), True)
])

def ingest():
  source_df = (spark.readStream
                      .format("cloudFiles")
                      .option("cloudFiles.format", "csv")  
                      .option("header", "true") 
                      .option("timestampFormat","yyyy-MM-dd")                  
                      .option("cloudFiles.schemaLocation", "dbfs:/FileStore/capstone/chekpoint/schema")
                      .option("cloudFiles.inferColumnTypes", "true")                    
                      .load("dbfs:/FileStore/capstone/transactions")
  )

  write_query = (source_df.writeStream
                          .format("delta")
                          .option("checkpointLocation", "dbfs:/FileStore/capstone/chekpoints/")
                          .option("mergeSchema", "true")
                          .outputMode("append")                         
                          .toTable("bronze.transactions")
  )

ingest()

In [0]:
%sql
select * from bronze.transactions;

transaction_id,customer_id,branch_id,channel,transaction_type,amount,currency,timestamp,status,_rescued_data
T6050,C1917,B0004,web,transfer,43.51,GBP,2023-01-22T07:39:00Z,completed,null
T6051,C1988,B0009,mobile,payment,8.08,EUR,2023-01-22T08:32:00Z,completed,null
T6052,C1186,B0007,mobile,deposit,14.01,USD,2023-01-22T09:04:00Z,completed,null
T6053,C1098,B0007,branch,deposit,67.58,EUR,2023-01-22T09:24:00Z,completed,null
T6054,C1804,B0008,ATM,deposit,24.75,USD,2023-01-22T09:45:00Z,completed,null
T6055,C1546,B0003,web,payment,28.34,EUR,2023-01-22T10:27:00Z,completed,null
T6056,C1416,B0009,branch,payment,35.45,GBP,2023-01-22T11:01:00Z,pending,null
T6057,C1859,B0008,mobile,transfer,25.13,GBP,2023-01-22T11:54:00Z,completed,null
T6058,C1388,B0009,branch,deposit,8.45,USD,2023-01-22T12:16:00Z,completed,null
T6059,C1332,B0000,mobile,deposit,52.64,GBP,2023-01-22T13:11:00Z,completed,null
